# Imagenet 1000 catagories

In [ ]:
import tensorflow as tf
from IPython.display import Image, display

def get_class(image_path):                
    display(Image(filename=image_path))
    
    # Read in the image_data
    image_data = tf.gfile.FastGFile(image_path, 'rb').read()

    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line
                   in tf.gfile.GFile("/notebooks/coscup/imagenet-1000/retrained_labels.txt")]

    # Unpersists graph from file
    with tf.gfile.FastGFile("/notebooks/coscup/imagenet-1000/retrained_graph.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

    with tf.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

        predictions = sess.run(softmax_tensor,
                               {'DecodeJpeg/contents:0': image_data})

        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]

        for node_id in top_k:
            human_string = label_lines[node_id]
            score = predictions[0][node_id]
            print('%s (score = %.5f)' % (human_string, score))


## 準備資料

In [ ]:
import os
import random

# read human readable lables
labels={}
with open("/notebooks/coscup/imagenet-1000/cls-loc-labels.txt") as f:
    for line in f:
        cols = line.split(",")
        labels[cols[0]] = line

        
# random query test image
filesList = []
for dirPath, dirNames, fileNames in os.walk("/notebooks/coscup/imagenet-1000/photos_test"):
    filesList += [os.path.join(dirPath, f) for f in fileNames if ".@__thumb" not in os.path.join(dirPath, f)] 

## Run Classifier

In [ ]:
target_file = random.choice(filesList)
category = target_file.split('/')[-2:][0]

print target_file
print labels[category]

get_class(target_file)